In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [120]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.5, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

In [6]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

class ReviewSummary(BaseModel):
    title: str
    bullets : list[str] = Field(min_length=3, max_length=5)
    tone : str

parser = PydanticOutputParser(pydantic_object=ReviewSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewSummary'>)

In [38]:
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"items": {"type": "string"}, "maxItems": 5, "minItems": 3, "title": "Bullets", "type": "array"}, "tone": {"title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
        [조건]
        오직 JSON 문자열만 출력을 해라.
        코드블록 금지.
        bullets의 요소는 단어.

        [출력양식]
        {fmt}
    """),
    ("user", 
     """
     다음 텍스트를 의사 결정용으로 요약: 
     {text}
     """)
]
).partial(fmt = fmt)
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "isTeam": {"title": "Isteam", "type": "boolean"}, "skills": {"description": "내가 사용한 기술", "items": {"type": "string"}, "title": "Skills", "type": "array"}, "implementationDetail": {"description": "나의 담당 구현 내용", "items": {"type": "string"}, "maxItems": 5, "minItems": 4, "title": "Implementationdetail", "type": "array"}, "result": {"description": "주요 성과/결과(실패한 결과도 가능) 맟 배운 점", "items": {"typ

In [98]:
chain = prompt | model | parser
chain

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "isTeam": {"title": "Isteam", "type": "boolean"}, "skills": {"description": "내가 사용한 기술", "items": {"type": "string"}, "title": "Skills", "type": "array"}, "implementationDetail": {"description": "나의 담당 구현 내용", "items": {"type": "string"}, "maxItems": 5, "minItems": 4, "title": "Implementationdetail", "type": "array"}, "result": {"description": "주요 성과/결과(실패한 결과도 가능) 맟 배운 점", "items": {"typ

In [45]:
text = """
    언어 공부하기는 좋은 앱이지만 영어 외에 다른 언어 공부하려면 영어를 잘해야 되는게 문제입니다. 저는 현재 일본어 학습자인데, 섹션 2부터 모르는 영어단어인 tempura, spacious, well-lit이 나와서 파x고를 돌려야하는 번거로움이 있었습니다. 또한 무과금들의 젬 수급이 너무 힘듭니다. 이제 퀘스트를 하면 젬 대신 물약이 나오고 젬 수급처는 프렌드 퀘스트랑 시간제한 퀘스트인데, 프렌드 퀘스트는 그냥 안하는 놈들이 있고 시간제한은 짬이 안나면 하기 힘듭니다. 광고가 너무 많은것도 문제입니다. 물론 돈벌려고 이 앱을 만들었을테니 광고가 나오는건 자연스러운 현상이지만, 한 레슨이 끝나고 광고가 두번씩이나(다른 앱 광고 1번, 앱 자체 현질유도 광고) 나오는건 심각해보입니다. 또한 출석제도는 한 레슨을 끝내야 이어갈 수 있는데, 하트가 다 줄어서 레슨 진행이 불가능(예:3개 있었는데 지문 3개를 틀려 하트 소멸)한 경우도 있는데 한문제라도 풀면 출석인 제도로 바꿔주세요.
    보석적용 버그, 오역있었고 점수가 만점시 80점 들어온다고 하면 보너스시간 적용하고 다맞춰도 뜬금없이 45점 들어옴. 그것도 줬다 안줬다ㅋㅋ 버그가 있는데 성적순으로 줄세우고 있음 피드백 제출해도 뭐 똑같은데... 주류인 영어는 많은데 일본어는 업데이트가 안되네요. 챕터3개가 끝이네요.. 환불받고 싶어요 as가 안좋아요
    안녕하세요 듀오링고를 약 270일간 잘 사용하고 있던 유저입니다. 최근 하트에서 에너지로 변화가 생겼는데, 다시 하트로 바꾸거나 에너지를 충전하고 소진하는 방식을 바꿔주셨으면 좋겠습니다. 듀오링고에선 학습을 촉진하기 위해서 에너지 체계로 바꿨다고 하였으나, 실수를 한번도 하지 않아도 레슨마다 기본적으로 깎이는 에너지가 생겨서 서너개밖에 못하고 에너지가 바닥나게 되더군요. 레슨마다 광고 한두개씩 붙는 것도 신경은 쓰였지만 참을만 했습니다. 하지만 이 업데이트는 제가 보기엔 정말 대놓고 돈쓰게 하려고 바꾼 걸로밖에 안보입니다. 진심으로 학습을 촉진하고 싶었으면 실수가 없었던 레슨은 문제를 풀 때마다 소진은 되더라도 그만큼 충전해주어서 최종적으론 에너지가 닳지 않아야 하는 거 아닙니까? 듀오링고의 목적은 무료로 언어를 배울 수 있게 하는 것이라고 광고하던데, 이번 업데이트는 그 목적과 많이 어긋나 보입니다. 전에는 정말 잘 쓰고 있던 유저로서 꼭 시정해주셨으면 합니다. 감사합니다.
    """

try:
    result = chain.invoke({
        "text" : text
    })
except ValidationError as e:
    print("ValidationError: ", e)

In [37]:
print(type(result))
result.model_dump()

<class '__main__.ReviewSummary'>


{'title': '듀오링고 앱 문제점 및 개선 요청',
 'bullets': ['언어장벽', '젬수급어려움', '과도한광고', '출석제도불편', '버그및오역'],
 'tone': '비판적'}

---

# 실습 1 : 포트폴리오 작성

In [153]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

class PotfolioSummary(BaseModel):
    serviceTitleRecommand: list[str] = Field(description="창의적인 후킹 서비스명 추천", min_length=3)
    title: str = Field(description="프로젝트 제목")
    isTeam: bool
    skills: list[str] = Field(description="내가 사용한 기술")
    implementationDetail: list[str] = Field(description="나의 담당 구현 내용", min_length=4, max_length=5)
    result: list[str] = Field(description="주요 성과/결과(실패한 결과도 가능)", min_length=2, max_length=3)
    lessonAndLearn : list[str] = Field(description="배운점", min_length=2, max_length=3)
    improvements : list[str] = Field(description="결과 이후 개선 사항", min_length=2, max_length=3)

parser = PydanticOutputParser(pydantic_object=PotfolioSummary)
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"serviceTitleRecommand": {"description": "창의적인 후킹 서비스명 추천", "items": {"type": "string"}, "minItems": 3, "title": "Servicetitlerecommand", "type": "array"}, "title": {"description": "프로젝트 제목", "title": "Title", "type": "string"}, "isTeam": {"title": "Isteam", "type": "boolean"}, "skills": {"description": "내가 사용한 기술", "items": {"type": "string"}, "title": "Skills", "type": "array"}, "implementationDetail": {"description": "나의 담당 구현 내용", "items": {"type": "string"}, "maxItems": 5, "minItems": 4, "title": "Implementationdet

In [154]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
        너는 15년차 AI 서비스 CTO 이면서 인사 담당자야
        
        [조건]
        서비스명은 창의적으로 작성.
        구체적인 예시를 사용하여 문장의 신뢰도를 올릴 것.
        말투는 포트폴리오에 작성 가능하게 명사형으로 끝낼 것.
        오직 JSON 문자열만 출력을 해라.
        코드블록 금지.

        [출력양식]
        {fmt}
    """),
    ("user", 
     """
     다음 내용을 포트폴리오 양식에 맞게 출력 :
     {text}
     """)
]
).partial(fmt = fmt)
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"serviceTitleRecommand": {"description": "창의적인 후킹 서비스명 추천", "items": {"type": "string"}, "minItems": 3, "title": "Servicetitlerecommand", "type": "array"}, "title": {"description": "프로젝트 제목", "title": "Title", "type": "string"}, "isTeam": {"title": "Isteam", "type": "boolean"}, "skills": {"description": "내가 사용한 기술", "items": {"type": "string"}, "title": "Skills", "type": "array"}, "implementationDetail": {"description": "나의 담당 구현 내용",

In [155]:
chain = prompt | model | parser
chain

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"serviceTitleRecommand": {"description": "창의적인 후킹 서비스명 추천", "items": {"type": "string"}, "minItems": 3, "title": "Servicetitlerecommand", "type": "array"}, "title": {"description": "프로젝트 제목", "title": "Title", "type": "string"}, "isTeam": {"title": "Isteam", "type": "boolean"}, "skills": {"description": "내가 사용한 기술", "items": {"type": "string"}, "title": "Skills", "type": "array"}, "implementationDetail": {"description": "나의 담당 구현 내용",

In [156]:
text = """
# 주제 선정 배경

## 수어의 필요성

- 수어가 왜 필요하지?
    - 선천적인 청각 장애인의 경우 수화가 더 편할 수 있음.
        - https://mch.nocutnews.co.kr/news/6136450
        - https://youtu.be/7DQRwW1rdmM
    - **수어를 모국어**처럼 배우는 경우가 많음.
    - https://mediin.or.kr/front/info/002/notice-detail.do?data_Id=2967&pageNum=1&searchText=&keyField=&type=center&codeId=C001&codeId2=002

# 수어 교육 (KSL)

레퍼런스 : 엔비디아 https://signs-ai.com/

임시 화면 구성

![image.png](attachment:40fef3a5-0166-4f8f-9ab2-6fde815adc71:image.png)

### Learning ASL(American Sign Language) → Learning KSL(Korean Sign Language)

- 한국어 수어를 배울 수 있도록 수어를 교육하는 서비스를 만든다.
- 마네퀸이 한국어 수어를 단어별로 보여주고, 사용자가 카메라를 기준으로 수어를 진행한다,
- 해당 모션이 맞는지 AI가 판단한다.

# 서비스 플로우

1. 언리얼, 웹캠 띄운다.
2. 소켓 통신으로 언리얼에서 웹캠 쪽으로 시작한다라는 신호를 보내준다. 
3. HELLO 같은 문제를 준다.
4. 그 문제를 보고 웹 캠 내에서 사용자가 모션을 한다.
5. 그 모션이 맞는 지 모델로 체크한다. 
6. 맞았으면 다음으로 넘어가라는 신호를  준다. (ml모델 → 언리얼)
7. 언리얼에서 다음 문제를 낸다.

### 주고 받을 데이터

- Request → 단어 key 값(sign_id), 이미지(images)
- Response → 단어 key값(sign_id), 정답 여부(is_correct), 통신 성공 여부(code)
- 형태 Json
    
    ```html
    Request : {"images" : image_bytes, "sign_id" : sign_id}
    Response : {"is_correct" : True(정답)/False(오답), "sign_id" : sign_id, "code" : 200(성공)/400(에러)}
    ```
    
- 사진을 주기적으로 보낼건지
    - 레디?

## **DataTable**

| label (int) | sign_num (int) | seq_Id (int) | sign_text  (string) | seq_count (int) | 손가락 좌표 42 column (float) | 포즈 좌표 4 column (float) |
| --- | --- | --- | --- | --- | --- | --- |
| 0 | 1 | 0 | 1 | 1 |  |  |
| 1 | 2 | 0 | 2 | 1 |  |  |
| 2 | 3 | 0 | 3 | 1 |  |  |
| 3 | 4 | 0 | 4 | 1 |  |  |
| 4 | 5 | 0 | 5 | 1 |  |  |
| 5 | 6 | 0 | 6 | 1 |  |  |
| 6 | 7 | 0 | 7 | 1 |  |  |
| 7 | 8 | 0 | 8 | 1 |  |  |
| 8 | 9 | 0 | 9 | 1 |  |  |
| 9 | 10 | 0 | 10 | 2 |  |  |
| 10 | 10 | 1 | 10 | 2 |  |  |
| 11 | 11 | 0 | 예 | 1 |  |  |
| 12 | 12 | 0 | 아니오 | 2 |  |  |
| 13 | 12 | 1 | 아니오 | 2 |  |  |
| 14 | 13 | 0 | 좋아요 | 1 |  |  |
| 15 | 14 | 0 | 싫어요 | 1 |  |  |
| 16 | 15 | 0 | 맞다 | 1 |  |  |
| 17 | 16 | 0 | 틀리다 | 2 |  |  |
| 18 | 16 | 1 | 틀리다 | 2 |  |  |
| 19 | 17 | 0 | 쉽다 | 2 |  |  |
| 20 | 17 | 1 | 쉽다 | 2 |  |  |
| 21 | 18 | 0 | 어렵다 | 2 |  |  |
| 22 | 18 | 1 | 어렵다 | 2 |  |  |
| 23 | 19 | 0 | 있어요 | 1 |  |  |
| 24 | 20 | 0 | 없어요 | 1 |  |  |

# 나의 주 담당 업무
자동화 데이터 수집 시스템: MediaPipe + OpenCV 기반 손/얼굴 랜드마크 자동 추출 시스템 구축
데이터 수집 : 11개 수어 단어(숫자 1-7, 좋아요/싫어요/맞다/틀리다)에 대해 데이터 수집 진행 (개 당 500개로 총 5500개)
데이터 전처리: 가이드박스 좌표 제약, 다양한 각도(왼쪽/오른쪽, 위/아래) 데이터 증강, 대비 효과 적용 전략 수립
모델 개발 및 최적화: XGBoost 기반 수어 분류 모델 개발, 7차례 피처 엔지니어링 반복으로 인식률 향상

# 데이터 수집

## 1차 데이터 수집

- 각 수어별로 얼굴과 손의 가이드 박스 생성
- 가이드 박스에 맞추어 데이터 수집 진행

## 2차 데이터 수집

- 가이드 박스 제한을 줄여, 가이드 박스 주변의 여러 좌표에서 수집 진행
- 손의 여러 각도로 수집 진행 (왼쪽/오른쪽, 위/아래)

데이터 수집 과정에서 특정 동작들의 경우 랜드마크를 제대로 인식 못하는 문제가 있었음

(다섯 손가락이 제대로 보이지 않는 경우 발생)

수집 과정에서 어두운 공간에서 손을 더 잘 인식하는 것을 발견

예상 : 배경과 손 간의 대비가 높을 수록 인식이 잘 됨

이후 랜드마크 데이터 추가 수집 시 

이미지에 대비 효과 적용 후 랜드마크 추출 방식 사용 예정

# 모델 선정

머신러닝 분류 모델 XGBoost 사용

# 모델 학습 및 평가

성능은 항상 좋음

성능이 좋지만 실제 예측은 잘 못하는 문제가 있었음

# 성능 향상을 위한 과정

## 1차 : 포인트 (손가락 위치 좌표)

가이드 박스로 포인트 수집

### 결과

1, 2, 3 분류 조차 못함

### 원인 분석

너무 정제된 데이터

### 해결 방법

데이터의 다양성을 증대 및 데이터 추가

약간 예측 성공 증가 매우 조금

## 2차 : 앵글 (손가락 마디 간의 각도)

손가락 마디들의 벡터를 구하고,

벡터 간의 코사인 계산을 통해  각도를 구한다

### 결과

숫자 1 -4까지 가능

5, 6, 7부터 구분 안됨

5 → 1

6 → 2

7 → 3

### 원인

2 3과 비슷한 손가락을 사용하기 때문에

손가락의 앵글이 비슷해서 헷갈릴 가능성이 높다고 판단

### 해결 방법

각 손가락 마디 간의 유닛 벡터를 구하여 손가락의 방향 정보를 추가

## 3차 : 앵글 + 벡터 (각 손가락 마디 간의 유닛 벡터)

### 결과

1- 7까지 구분 가능

하지만, 예, 좋아요 추가 후

예 ↔  7

5 ↔  좋아요

헷갈려함

### 원인

예(엄지, 검지, 중지, 새끼) ↔  7(검지, 중지, 약지) > 새끼 손가락과 엄지 손가락의 구부러짐에 대한 데이터 수집이 제대로 되지 않음

5 ↔  좋아요 > 손 바닥/손날 이 화면을 보는 그 각도를 구분하지 못함

### 해결 방법


- 빨간색으로 표시한 손바닥의 방향성에 대한 피처를 추가해보자
- 각 동작별로 얼굴과 거리가 다름 > 얼굴과 손 간의 거리에 대한 벡터를 추가해보자

## 4차 : 앵글 + 벡터(curv 추가) + 페이스 거리

### 결과

5번 ↔ 좋아요 헷갈려함

4번 ↔ 맞다를 헷갈려함

싫어요 → 2번으로 인식

틀려요 → 7번으로 인식(가끔 2번)

엄지가 나오지 않는 엄지가 접힌 케이스들의 인식이 제대로 되지 않고 있음

### 해결 방법

- 5번 뒤로 꺽은 데이터 추가 필요

## 5차 : 앵글 + 벡터(curv 추가) + 페이스 거리(nose, wrist)

페이스와의 거리에 대한 피쳐가 너무 많은 것 같다는 의견

코, 손목 간의 거리 벡터만 추가하는 것으로 축소

→ 약간의 성능 향상 있었음

1-3 분류 잘함 (손가락을 뒤로 좀 젖히면 잘하는 느낌)

4 는 맞다로 인식

5 의 경우 손을 앞으로 말아야 잘하는 느낌


싫어요 6으로 인식 (동일 손가락을 사용하고 각도가 비슷함)

틀려요 2로 인식 (동일 손가락을 사용하고 각도가 비슷함)

## 6차 : 맞다, 좋아요, 싫어요 분류 모델

세 가지를 매우 잘 구분

## 최종 결론

숫자와 단어 데이터를 분리해서 서비스를 완성하는 방향으로 

### 이후 방향성

- 손의 돌아간 정도를 분류 할 수 있는 데이터 추가가 필요
- 더 정확한 데이터 수집 필요 (엄지, 새끼 손가락) - 대비 효과 사용
- 팔의 각도도 데이터 수집 시 추가

## 7차 : 아직 진행 중…

- 손가락과 손목간의 거리
- 손가락 포인트 간의 거리
"""

try:
    result = chain.invoke({
        "text" : text
    })
except ValidationError as e:
    print("ValidationError: ", e)

In [157]:
result.model_dump()

{'serviceTitleRecommand': ['SignaLearn', 'HandSpeak AI', 'KSL Mastery'],
 'title': 'KSL AI 기반 수어 교육 서비스 개발',
 'isTeam': True,
 'skills': ['Python',
  'MediaPipe',
  'OpenCV',
  'XGBoost',
  'Unreal Engine',
  'Socket 통신',
  '데이터 증강',
  '피처 엔지니어링'],
 'implementationDetail': ['MediaPipe와 OpenCV를 활용한 손과 얼굴 랜드마크 자동 추출 시스템 구축',
  '11개 수어 단어에 대해 500개씩 총 5500개 데이터 수집 및 전처리 수행',
  '가이드박스 좌표 제약과 다양한 각도 데이터 증강 및 대비 효과 적용 전략 수립',
  'XGBoost 기반 수어 분류 모델 개발 및 7차례 피처 엔지니어링 반복으로 인식률 향상',
  '언리얼 엔진과 웹캠 간 소켓 통신 구현, AI 모델과의 실시간 문제 출제 및 정답 판단 로직 개발'],
 'result': ['XGBoost 모델을 통한 수어 단어 인식률 크게 향상, 특히 숫자 1~7 및 기본 단어 구분 성공',
  '실제 예측에서 발생하는 혼동 문제 발견 및 개선 방향 도출',
  '서비스 플로우 기반 실시간 수어 학습 및 평가 시스템 프로토타입 완성'],
 'lessonAndLearn': ['정제된 데이터만으로는 모델 성능 한계 존재, 데이터 다양성과 증강의 중요성 인지',
  '손가락 각도 및 벡터 기반 피처가 인식 성능 향상에 핵심 역할 수행',
  '실제 환경에서의 손 위치, 각도, 얼굴과의 거리 등 다양한 변수 반영 필요성 학습'],
 'improvements': ['손바닥 방향성 및 얼굴과 손 간 거리 벡터 추가로 혼동 단어 구분 성능 개선 계획',
  '대비 효과 적용 등 더 정교한 데이터 수집 전략 수립 예정',
  '팔 각도 및 손가락과 손목 간 거리 등 추가 피처 수집 및 모델 

---

# 실습 2 : 허깅페이스 모델 요약

In [144]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser
from typing import Any


class ItBlogSummary(BaseModel):
    title: str
    feature: str = Field(description="주요 기능에 대해서 한 문장 요약")
    skill: str = Field(description="핵심 기술 한 문장 요약")
    skilldetails: dict[str, Any] = Field(description="각 기술에 대한 설명")
    addition : list[str] = Field(description="추가로 공부하면 좋은 내용 또는 링크", max_length=3)

parser = PydanticOutputParser(pydantic_object=ItBlogSummary)
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "feature": {"description": "주요 기능에 대해서 한 문장 요약", "title": "Feature", "type": "string"}, "skill": {"description": "핵심 기술 한 문장 요약", "title": "Skill", "type": "string"}, "skilldetails": {"additionalProperties": true, "description": "각 기술에 대한 설명", "title": "Skilldetails", "type": "object"}, "addition": {"description": "추가로 공부하면 좋은 내용 또는 링크", "items": {"type": "string"}, "maxItems": 3, "title": "Addition", "type": "array"}}, "required": ["title", "feature", "skill", "skilldetail

In [145]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
        너는 15년차 AI 서비스 CTO야
        
        [조건]
        한글로 작성할 것
        오직 JSON 문자열만 출력을 해라.
        코드블록 금지.

        [출력양식]
        {fmt}
    """),
    ("user", 
     """
     다음 내용을 포트폴리오 양식에 맞게 출력 :
     {text}
     """)
]).partial(fmt = fmt)
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "feature": {"description": "주요 기능에 대해서 한 문장 요약", "title": "Feature", "type": "string"}, "skill": {"description": "핵심 기술 한 문장 요약", "title": "Skill", "type": "string"}, "skilldetails": {"additionalProperties": true, "description": "각 기술에 대한 설명", "title": "Skilldetails", "type": "object"}, "addition": {"description": "추가로 공부하면 좋은 내용 또는 링크", "items": {"type": "string"}, "maxItems": 3, "title"

In [134]:
text = """
Qwen3-VL-235B-A22B-Instruct
Meet Qwen3-VL — the most powerful vision-language model in the Qwen series to date.

This generation delivers comprehensive upgrades across the board: superior text understanding & generation, deeper visual perception & reasoning, extended context length, enhanced spatial and video dynamics comprehension, and stronger agent interaction capabilities.

Available in Dense and MoE architectures that scale from edge to cloud, with Instruct and reasoning‑enhanced Thinking editions for flexible, on‑demand deployment.

Key Enhancements:
Visual Agent: Operates PC/mobile GUIs—recognizes elements, understands functions, invokes tools, completes tasks.

Visual Coding Boost: Generates Draw.io/HTML/CSS/JS from images/videos.

Advanced Spatial Perception: Judges object positions, viewpoints, and occlusions; provides stronger 2D grounding and enables 3D grounding for spatial reasoning and embodied AI.

Long Context & Video Understanding: Native 256K context, expandable to 1M; handles books and hours-long video with full recall and second-level indexing.

Enhanced Multimodal Reasoning: Excels in STEM/Math—causal analysis and logical, evidence-based answers.

Upgraded Visual Recognition: Broader, higher-quality pretraining is able to “recognize everything”—celebrities, anime, products, landmarks, flora/fauna, etc.

Expanded OCR: Supports 32 languages (up from 19); robust in low light, blur, and tilt; better with rare/ancient characters and jargon; improved long-document structure parsing.

Text Understanding on par with pure LLMs: Seamless text–vision fusion for lossless, unified comprehension.

Model Architecture Updates:



Interleaved-MRoPE: Full‑frequency allocation over time, width, and height via robust positional embeddings, enhancing long‑horizon video reasoning.

DeepStack: Fuses multi‑level ViT features to capture fine‑grained details and sharpen image–text alignment.

Text–Timestamp Alignment: Moves beyond T‑RoPE to precise, timestamp‑grounded event localization for stronger video temporal modeling.

This is the weight repository for Qwen3-VL-235B-A22B-Instruct.

Model Performance
Multimodal performance



Pure text performance


Quickstart
Below, we provide simple examples to show how to use Qwen3-VL with 🤖 ModelScope and 🤗 Transformers.

The code of Qwen3-VL has been in the latest Hugging Face transformers and we advise you to build from source with command:

pip install git+https://github.com/huggingface/transformers
# pip install transformers==4.57.0 # currently, V4.57.0 is not released

Using 🤗 Transformers to Chat
Here we show a code snippet to show how to use the chat model with transformers:

from transformers import Qwen3VLMoeForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-235B-A22B-Instruct", dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLMoeForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-235B-A22B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-235B-A22B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

Citation
If you find our work helpful, feel free to give us a cite.

@misc{qwen2.5-VL,
    title = {Qwen2.5-VL},
    url = {https://qwenlm.github.io/blog/qwen2.5-vl/},
    author = {Qwen Team},
    month = {January},
    year = {2025}
}

@article{Qwen2VL,
  title={Qwen2-VL: Enhancing Vision-Language Model's Perception of the World at Any Resolution},
  author={Wang, Peng and Bai, Shuai and Tan, Sinan and Wang, Shijie and Fan, Zhihao and Bai, Jinze and Chen, Keqin and Liu, Xuejing and Wang, Jialin and Ge, Wenbin and Fan, Yang and Dang, Kai and Du, Mengfei and Ren, Xuancheng and Men, Rui and Liu, Dayiheng and Zhou, Chang and Zhou, Jingren and Lin, Junyang},
  journal={arXiv preprint arXiv:2409.12191},
  year={2024}
}

@article{Qwen-VL,
  title={Qwen-VL: A Versatile Vision-Language Model for Understanding, Localization, Text Reading, and Beyond},
  author={Bai, Jinze and Bai, Shuai and Yang, Shusheng and Wang, Shijie and Tan, Sinan and Wang, Peng and Lin, Junyang and Zhou, Chang and Zhou, Jingren},
  journal={arXiv preprint arXiv:2308.12966},
  year={2023}
}
"""

In [146]:
chain = prompt | model | parser

try:
    result = chain.invoke({
        "text" : text
    })
except ValidationError as e:
    print("ValidationError: ", e)

In [147]:
result.model_dump()

{'title': 'Qwen3-VL-235B-A22B-Instruct',
 'feature': 'Qwen3-VL은 텍스트와 비전을 통합하여 고도화된 시각-언어 이해, 공간 및 영상 추론, 확장된 컨텍스트 처리 능력을 갖춘 최첨단 비전-언어 모델입니다.',
 'skill': '멀티모달 AI, 고해상도 영상 및 텍스트 통합 처리, 장기 컨텍스트 및 시공간 추론 기술을 핵심으로 합니다.',
 'skilldetails': {'Visual Agent': 'PC 및 모바일 GUI 요소 인식과 도구 호출을 통한 작업 완수를 지원하는 시각 에이전트 기능.',
  'Visual Coding Boost': '이미지 및 영상으로부터 Draw.io, HTML, CSS, JS 코드 자동 생성.',
  'Advanced Spatial Perception': '객체 위치, 시점, 가림 현상 판단 및 2D/3D 공간 추론을 통한 고도화된 공간 인지.',
  'Long Context & Video Understanding': '최대 256K 기본 컨텍스트, 1M 확장 가능, 수시간 분량 영상과 책을 완전 기억 및 세컨드 레벨 인덱싱 처리.',
  'Enhanced Multimodal Reasoning': 'STEM 및 수학 문제에 특화된 인과 분석과 논리적, 증거 기반 답변 제공.',
  'Upgraded Visual Recognition': '셀럽, 애니메이션, 제품, 랜드마크, 동식물 등 광범위한 사전학습으로 모든 것을 인식.',
  'Expanded OCR': '32개 언어 지원, 저조도·흐림·기울기 환경 강인, 희귀문자 및 장문 구조 분석 성능 향상.',
  'Text Understanding': '순수 LLM에 준하는 텍스트 이해력과 텍스트-비전 융합으로 손실 없는 통합 이해.',
  'Interleaved-MRoPE': '시간, 너비, 높이 전 주파수 위치 임베딩을 통한 장기 영상 추론 강화.',
  'DeepStack': '다중 수준 ViT 특징 융합으로 세밀한 이미지-텍스트 